In [113]:
import re
from nltk.tokenize import word_tokenize, sent_tokenize, regexp_tokenize 
from matplotlib.pyplot import hist, show
from collections import Counter
import urllib
import codecs
from nltk.stem import WordNetLemmatizer

## Czytania danego dnia

In [132]:
#downloading the webpage
import requests
import datetime

now = datetime.datetime.now()
year = str(now.year)
month = str(now.month)
if len(month) == 1:
    month = "0" + month
day = str(now.day)
ymd = year+month+day
address = "http://mateusz.pl/czytania/" + year + "/" + ymd + ".html"
page = requests.get(address)

### Lemmatyzacja (lemmatisation) 
Algorytm znajdowania lemmy, formy podstawowej wyrazu w obszarze części mowy którą reprezentuje. Inaczej: wyszukiwanie formy kanonicznej leksemu. W komputerowej analizie języka naturalnego lemmatyzacja jest operacją bardziej precyzyjną niż stemmatyzacja. 

### Stemmatyzacja (stemming) 
Sprowadzenie wyrazu reprezentującego dowolną część mowy do stemu, rdzenia postaci źródłowej, najczęściej rzeczownika, bądź niekiedy nawet sekwencji znaków nie mającej samodzielnego znaczenia w języku naturalnym. 

In [79]:
polish_stops = codecs.open("polishStopWords",'r','utf-8')

In [80]:
ps = polish_stops.read().split('\r\n')

In [36]:
milosc = codecs.open("milosc",'r','utf-8')

In [100]:
milujcie = codecs.open("milujcie",'r','utf-8')

In [101]:
milujcie_sent = milujcie.read().split('\r\n')

In [105]:
song = ""
for sentence in milujcie_sent:
    song = song + sentence

In [99]:
text = milosc.read()
#text = song

In [106]:
tokens = word_tokenize(text)
lower_tokens = [t.lower() for t in tokens]

### Lemmatization

In [114]:
# Retain alphabetic words: alpha_only
alpha_only = [t for t in lower_tokens if t.isalpha()]

In [115]:
# Remove all stop words: no_stops
no_stops = [t for t in alpha_only if t not in ps]

In [116]:
#bag of words
bow = Counter(no_stops)

In [117]:
bow.most_common(20)

[('miłość', 9),
 ('bym', 4),
 ('miał', 4),
 ('gdybym', 3),
 ('miłości', 3),
 ('szuka', 2),
 ('unosi', 2),
 ('miłujcie', 1),
 ('wzajemnie', 1),
 ('umiłowałem', 1),
 ('mówił', 1),
 ('językami', 1),
 ('ludzi', 1),
 ('aniołów', 1),
 ('miałstałbym', 1),
 ('miedź', 1),
 ('brzęcząca', 1),
 ('cymbał', 1),
 ('brzmiący', 1),
 ('dar', 1)]